In [ ]:
import dask.dataframe as dd


In [ ]:
df = dd.read_parquet("data/icesat2_dem_filtered_fixed_1.parquet")


In [ ]:
for x in  df.columns:
    print(x)


In [ ]:
import duckdb

duckdb.sql("""
    COPY (
        SELECT * EXCLUDE geometry FROM 'data/icesat2_dem.parquet'
        WHERE atl03_cnf = 4 AND atl08_class = 1
    )
    TO 'data/icesat2_dem_filtered.parquet' (FORMAT PARQUET)
""")


In [ ]:
import duckdb

# 1. Відкриваємо Parquet як DuckDB-таблицю
duckdb.sql("""
    CREATE OR REPLACE TABLE temp_table AS
    SELECT
        *,
        "copernicus_dеm_slope" AS "copernicus_dem_slope"
    FROM 'data/icesat2_dem_filtered.parquet'
""")

# 2. Видаляємо стару колонку з кирилицею (опціонально)
duckdb.sql("""
    ALTER TABLE temp_table DROP COLUMN "copernicus_dеm_slope"
""")

# 3. Зберігаємо як новий Parquet-файл з правильними назвами колонок
duckdb.sql("""
    COPY temp_table TO 'data/icesat2_dem_filtered_fixed.parquet' (FORMAT PARQUET)
""")


In [ ]:
import pandas as pd

# 1. Завантажуємо Parquet
df = pd.read_parquet("data/icesat2_dem_filtered.parquet")

# 2. Замінюємо всі кириличні “е” на латиничні “e” в усіх назвах колонок
df.columns = [col.replace("е", "e") for col in df.columns]

# 3. Зберігаємо новий файл
df.to_parquet("data/icesat2_dem_filtered_fixed_1.parquet")


In [ ]:
import pandas as pd
df = pd.read_parquet("data/icesat2_dem_filtered_fixed_1.parquet")
print(df["lulc_class"].unique())
print(df["copernicus_dem_landform"].unique())


In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "copernicus_dem"

# Унікальні LULC класи
print(duckdb.sql(f"SELECT DISTINCT lulc_name FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL").df())

# Унікальні landform класи
print(duckdb.sql(f"SELECT DISTINCT {dem}_landform FROM '{parquet_file}' WHERE {dem}_landform IS NOT NULL").df())


In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
duckdb.sql(f"PRAGMA table_info('{parquet_file}')").df()


In [ ]:
dem = "alos_dem"
sql = f"SELECT COUNT(*) as n FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL"
print(duckdb.sql(sql).df())


In [ ]:
sql = f"SELECT COUNT(*) as n FROM '{parquet_file}' WHERE {dem}_slope IS NOT NULL"
print(duckdb.sql(sql).df())


In [ ]:
sql = f"SELECT DISTINCT delta_{dem} FROM '{parquet_file}' LIMIT 10"
print(duckdb.sql(sql).df())

In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "alos_dem"
dff = duckdb.sql(f"SELECT delta_{dem} FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL LIMIT 20").df()
print(dff.head())
print(dff.shape)



In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "alos_dem"
print("Check if parquet readable and not empty...")
dff = duckdb.sql(f"SELECT delta_{dem} FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL").df()
print(dff)


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_box(y=dff[f"delta_{dem}"], name=f"delta_{dem}")
fig.show()
